<a href="https://colab.research.google.com/github/omararh/Expression-editor-assistant/blob/main/phi-3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
print("La version de torch est : ", torch.__version__)
print("Le calcul GPU est disponible ? ", torch.cuda.is_available())
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")


La version de torch est :  2.6.0+cu124
Le calcul GPU est disponible ?  True


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "microsoft/Phi-3-mini-4k-instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",   # ou torch.float16 / torch.bfloat16 si tu sais que c'est supporté
    device_map="auto"     # pour charger automatiquement sur le GPU si disponible
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [3]:
import json

def read_file(file_path):
  with open(file_path, 'r', encoding='utf-8') as f:
    return json.load(f)

INPUT_DATA_BASE_PATH = 'input_data/'

AVAILABLE_CONTEXT = read_file(INPUT_DATA_BASE_PATH + 'context.json')

SYMFONY_EXAMPLES = read_file(INPUT_DATA_BASE_PATH + 'examples.json')

In [4]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import spacy
!python -m spacy download fr_core_news_md

# Load french tokenizer, tagger, parser
nlp = spacy.load('fr_core_news_md')

def get_embedding(text):
    """
    Get normalized average embedding of all tokens (filtered).
    """
    doc = nlp(text)
    vectors = np.array([token.vector for token in doc if token.has_vector and not token.is_stop and not token.is_punct])

    if len(vectors) == 0:
        return np.zeros((nlp.vocab.vectors_length,))

    avg_vector = np.mean(vectors, axis=0)
    return avg_vector / np.linalg.norm(avg_vector)


def select_relevant_examples(user_query, examples_dict, top_k=5):
    """
    Sélectionne les exemples les plus pertinents en fonction de la similarité
    d'embedding avec la requête utilisateur.

    Args:
        user_query: user request
        examples_dict: (description -> expression) examples
        top_k: K relevant examples to be returned

    Returns:
        K relevant examples
    """
    query_embedding = get_embedding(user_query)

    similarities = []

    for description, expression in examples_dict.items():
        desc_embedding = get_embedding(description)

        similarity = cosine_similarity([query_embedding], [desc_embedding])[0][0]
        similarities.append((similarity, description, expression))

    similarities.sort(reverse=True)
    top_examples = [(desc, expr) for _, desc, expr in similarities[:top_k]]

    return top_examples


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 23.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
def generate_expression(user_intent, available_context=None, examples=None, max_retries=1):
    """
    Génère une suggestion d'expression Symfony avec retry en cas d'échec
    Args:
        user_intent (str): Intention ou question de l'utilisateur
        available_context (dict, optional): Contexte disponible dans l'éditeur
        examples (list, optional): Exemples d'expressions valides
        max_retries (int, optional): Nombre maximal de tentatives en cas d'erreur
    Returns:
        str: Expression Symfony suggérée par le modèle
    """
    # Construction du prompt pour Phi-2
    prompt = _build_prompt(user_intent, available_context, examples)

    retries = 0
    while retries <= max_retries:
        try:
            # Génération et traitement de la réponse
            response = _generate_and_process_response(prompt)

            # Vérification que la réponse n'est pas vide
            if _is_valid_response(response):
                return response
            else:
                # Si réponse vide ou invalide, retenter
                retries = _handle_retry(retries, max_retries, "Réponse vide ou invalide")
        except Exception as e:
            # En cas d'erreur, retenter
            retries = _handle_retry(retries, max_retries, f"Erreur lors de la génération: {e}")

    # Si après toutes les tentatives, rien n'a fonctionné
    return "Une erreur est survenue, veuillez réessayer."


def _build_prompt(user_intent, available_context, examples):
    """Construit le prompt pour le modèle de génération"""

    # Intro
    prompt = (
        "Tu es un assistant expert dans la création d'expressions Symfony Expression Language.\n"
        "Tu aides les utilisateurs à construire des expressions logiques simples, valides et efficaces selon la documentation :\n"
        "https://symfony.com/doc/current/reference/formats/expression_language.html\n\n"
    )

    # Contexte disponible (variables, méthodes, etc.)
    if available_context:
        prompt += "### CONTEXTE DISPONIBLE DANS L'ÉDITEUR ###\n"
        for category, items in available_context.items():
            prompt += f"- {category} : {', '.join(items)}\n"
        prompt += "\n"

    # Exemples pertinents
    if examples:
        prompt += "### EXEMPLES PERTINENTS ###\n"
        # Inverser les exemples pour avoir description -> expression
        inverted_examples = {value: key for key, value in examples.items()}
        relevant_examples = select_relevant_examples(user_intent, inverted_examples, top_k=5)
        for description, expression in relevant_examples:
            prompt += f"[Description] {description}\n"
            prompt += f"[Expression] {expression}\n\n"

    # Règles claires et strictes
    prompt += "### CONSIGNES À RESPECTER ###\n"
    prompt += "- Utilise uniquement la syntaxe du Symfony Expression Language\n"
    prompt += "- Utilise uniquement les variables, contextes et fonctions listés ci-dessus\n"
    prompt += "- Ne génère **aucune explication** ou texte inutile autour de l'expression\n"
    prompt += "- Simplifie l'expression au **strict minimum** nécessaire à la demande\n"
    prompt += "- Ne combine pas plusieurs conditions sauf si explicitement demandé (évite les AND, OR, NOT superflus)\n"
    prompt += "- Ne crée pas de fonctions ou méthodes non listées dans le contexte\n\n"

    # Intention de l'utilisateur
    prompt += "### DEMANDE DE L'UTILISATEUR ###\n"
    prompt += f"{user_intent.strip()}\n\n"

    # Plan d’analyse explicite (encourage une sortie claire)
    prompt += "Analyse étape par étape:\n"
    prompt += "1. Identifie les conditions ou actions explicitement demandées\n"
    prompt += "2. Choisis les fonctions/variables nécessaires dans le contexte\n"
    prompt += "3. Construis l'expression Symfony minimale correspondante\n\n"

    # Clôture
    prompt += "Expression Symfony: "

    return prompt

def _generate_and_process_response(prompt):
    """Génère une réponse et extrait la partie pertinente"""
    # Encoder le prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Minimized parameters
    outputs = model.generate(
        **inputs,
        max_new_tokens=50,         #
        do_sample=False,           #
        num_beams=1,               #
        top_p=None,                #
        eos_token_id=tokenizer.eos_token_id
    )

    # Décodage de la sortie
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extraction de la partie pertinente
    return _extract_response(full_response, prompt)


def _extract_response(full_response, prompt):
    """Extrait et nettoie la partie pertinente de la réponse"""
    # Extraction après "Expression Symfony: "
    if "Expression Symfony: " in full_response:
        response = full_response.split("Expression Symfony: ")[-1].strip()
    else:
        # Fallback au cas où le modèle ne respecte pas le format
        response = full_response.replace(prompt, "").strip()

    # Nettoyage final (retirer les explications qui pourraient suivre l'expression)
    if "\n" in response:
        response = response.split("\n")[0].strip()

    return response


def _is_valid_response(response):
    """Vérifie si la réponse est valide"""
    return response and not response.isspace() and "```" not in response


def _handle_retry(retries, max_retries, error_message):
    """Gère la logique de retry et renvoie le nouveau compteur"""
    retries += 1
    print(f"{error_message}, tentative {retries}/{max_retries+1}...")

    # Attendre un peu avant de retenter
    import time
    time.sleep(1)

    return retries

In [8]:
user_query_1 = "Pour chaque ligne du document, vérifier si le statut est 'pending'"
result_1 = generate_expression(user_query_1, AVAILABLE_CONTEXT, SYMFONY_EXAMPLES)
print("Résultat 1: Pour chaque ligne du document, vérifier si le statut est 'pending'")
print(result_1)

Résultat 1: Pour chaque ligne du document, vérifier si le statut est 'pending'
row.status == 'pending'
